In [1]:
# Colab-ячейка 1: устанавливаем зависимости и проверяем GPU
!pip install -q transformers datasets sentencepiece huggingface_hub

import torch
import transformers
import datasets

print("CUDA доступен?", torch.cuda.is_available())
print("Название устройства:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "–")
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("torch:", torch.__version__)


CUDA доступен? True
Название устройства: Tesla T4
transformers: 4.52.2
datasets: 2.14.4
torch: 2.6.0+cu124


In [2]:
# Colab-ячейка 2: импортируем библиотеки
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import math
import torch
from sklearn.model_selection import train_test_split


In [54]:
# Colab-ячейка 3: читаем CSV и разбиваем на train/validation
csv_path = "/content/dialog_bot_ai.csv"  # укажите путь к вашему файлу
df = pd.read_csv(csv_path, encoding="utf-8")

print(df.head())
print(f"Всего строк в исходном датасете: {len(df)}")

# Разбиваем на train и validation (90%/10%)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

print(f"Строк в train: {len(train_df)}, строк в validation: {len(val_df)}")

# Конвертируем в Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df.reset_index(drop=True))


                                            question  \
0  Салют! Что делать, если я потерял информацию о...   
1  SOS, что делать, если я потеряла данные к вход...   
2  Эй! Что делать если я потеряла данные, чтобы в...   
3  Срочно, я потеряла данные, чтобы войти в кабин...   
4  Эй! Я потеряла данные, чтобы войти в кабинет, ...   

                                              answer  
0  Пожалуйста, Чтобы восстановить учетные данные ...  
1  Чтобы восстановить учетные данные (login/parol...  
2  Чтобы восстановить учетные данные (login/parol...  
3  Здравствуйте! Чтобы восстановить учетные данны...  
4  Конечно, Чтобы восстановить учетные данные (lo...  
Всего строк в исходном датасете: 318
Строк в train: 286, строк в validation: 32


In [55]:
# Colab-ячейка 4: загружаем модель и токенизатор, добавляем спецтокены
model_name = "t-bank-ai/ruDialoGPT-small"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(model_name)

# Добавляем <User>: и <Bot>: как дополнительные специальные токены
additional_tokens = ["<User>:", "<Bot>:"]
tokenizer.add_special_tokens({"additional_special_tokens": additional_tokens})

# Если pad_token отсутствует, добавляем его как eos_token
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
model.resize_token_embeddings(len(tokenizer))

print("Размер словаря после добавления спецтокенов:", len(tokenizer))
print("Special tokens:", tokenizer.additional_special_tokens)


Размер словаря после добавления спецтокенов: 50264
Special tokens: ['<User>:', '<Bot>:']


In [56]:
# Colab-ячейка 5: токенизация с маскированием пары «prompt» vs «answer»
max_length = 256

def tokenize_and_mask(examples):
    questions = examples["question"]
    answers   = examples["answer"]
    input_ids_list = []
    attention_list = []
    labels_list    = []

    for q, a in zip(questions, answers):
        q_strip = q.strip()
        a_strip = a.strip()

        # Собираем один пример: <User>: Вопрос [EOS] <Bot>: Ответ [EOS]
        full_text = f"<User>: {q_strip} {tokenizer.eos_token} <Bot>: {a_strip} {tokenizer.eos_token}"

        # Токенизируем
        tok = tokenizer(
            full_text,
            truncation=True,
            max_length=max_length,
            padding="max_length"
        )
        input_ids      = tok["input_ids"]
        attention_mask = tok["attention_mask"]

        # Найдём индекс первого токена answer (первый токен после "<Bot>:")
        bot_token_id = tokenizer.convert_tokens_to_ids("<Bot>:")
        # Найдём позицию в input_ids, где встретился "<Bot>:"
        try:
            bot_pos = input_ids.index(bot_token_id)
        except ValueError:
            # на случай, если спецтокен почему-то не в словаре
            bot_pos = None

        # Формируем labels: всё до и включая "<Bot>:" → -100, всё после → input_id (или -100 если pad)
        labels = []
        for idx, tid in enumerate(input_ids):
            if bot_pos is not None and idx <= bot_pos:
                labels.append(-100)
            else:
                # если это padding → -100, иначе реальный id
                labels.append(tid if tid != tokenizer.pad_token_id else -100)

        input_ids_list.append(input_ids)
        attention_list.append(attention_mask)
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_list,
        "labels": labels_list
    }

# Применяем токенизацию к train и validation наборам
tokenized_train = train_dataset.map(
    tokenize_and_mask,
    batched=True,
    remove_columns=["question", "answer"]
)
tokenized_val = val_dataset.map(
    tokenize_and_mask,
    batched=True,
    remove_columns=["question", "answer"]
)


Map:   0%|          | 0/286 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [57]:
# Colab-ячейка 6: создаём DataCollator для каузальной модели
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [58]:
# Colab-ячейка 7: определяем функцию для вычисления метрик (перплексия)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Сдвигаем: прогнозы на i-м шаге соответствуют меткам на i+1
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    # Флэттенинг
    batch_size, seq_len, vocab_size = shift_logits.shape
    flat_logits = shift_logits.reshape(-1, vocab_size)
    flat_labels = shift_labels.reshape(-1)
    # Считаем лосс, игнорируя -100
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(flat_logits, flat_labels)
    ppl = math.exp(loss.item()) if loss.item() < 20 else float("inf")
    return {"perplexity": ppl}


In [59]:
# Colab-ячейка 8: задаём TrainingArguments с валидацией и сохранением по «эпохам»
import math

output_dir = "/content/ru_dialobot_gpu"

# Считаем шаги на одну «эпоху» для eval_steps и save_steps
train_size = len(tokenized_train)
effective_batch_size = 1 * 8  # per_device_train_batch_size=1, gradient_accumulation_steps=8
steps_per_epoch = math.ceil(train_size / effective_batch_size)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,

    # параметры обучения
    num_train_epochs=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    warmup_steps=50,

    # Валидация и сохранение «по эпохам»
    do_train=True,
    do_eval=True,
    eval_steps=steps_per_epoch,
    save_steps=steps_per_epoch,
    save_total_limit=2,

    logging_dir=f"{output_dir}/logs",
    logging_steps=100,

    report_to="none",
    fp16=True
)


In [60]:
# Colab-ячейка 9: создаём Trainer и запускаем обучение
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

train_output = trainer.train()
print("Результаты обучения:", train_output.metrics)


<ipython-input-60-9e50d2bd0993>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Результаты обучения: {'train_runtime': 66.6486, 'train_samples_per_second': 8.582, 'train_steps_per_second': 1.08, 'total_flos': 74729521152000.0, 'train_loss': 3.1323106553819446, 'epoch': 2.0}


In [61]:
# Colab-ячейка 10: сохраняем модель и токенизатор
trainer.save_model(output_dir)         # pytorch_model.bin + config.json
tokenizer.save_pretrained(output_dir)  # файлы токенизатора
torch.save(model.state_dict(), f"{output_dir}/model_weights.pt")

print(f"Все файлы сохранены в {output_dir}")


Все файлы сохранены в /content/ru_dialobot_gpu


In [39]:
!pip install -q huggingface_hub

In [62]:
# Colab-ячейка 11: установка huggingface_hub, авторизация и пуш модели
!pip install -q huggingface_hub

from huggingface_hub import notebook_login, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1) Вызываем notebook_login() и сохраняем возвращённый токен
hf_token = notebook_login()  # вставьте ваш Hugging Face токен при запросе

# 2) Передаём токен явно в whoami()
api = HfApi()
user_info = api.whoami(token=hf_token)
username = user_info["name"]

repo_name = "ruDialoGPT-finetuned"
full_repo_id = f"{username}/{repo_name}"

# 3) Создаём репозиторий (если ещё не существует)
api.create_repo(
    repo_id=full_repo_id,
    token=hf_token,
    exist_ok=True,
    private=False
)
print(f"Репозиторий создан (или уже существовал): https://huggingface.co/{full_repo_id}")

# 4) Загружаем локальную модель и токенизатор
model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# 5) Проверяем pad_token и при необходимости увеличиваем эмбеддинги
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

# 6) Пушим модель и токенизатор, передав токен явно
model.push_to_hub(repo_id=full_repo_id, use_auth_token=hf_token)
tokenizer.push_to_hub(repo_id=full_repo_id, use_auth_token=hf_token)

print(f"Модель и токенизатор запушены в https://huggingface.co/{full_repo_id}")


Репозиторий создан (или уже существовал): https://huggingface.co/Dilshodbek11/ruDialoGPT-finetuned


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Модель и токенизатор запушены в https://huggingface.co/Dilshodbek11/ruDialoGPT-finetuned


In [63]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
repo_id = "Dilshodbek11/ruDialoGPT-finetuned"

# 1) Загружаем fine-tuned модель и токенизатор
model     = AutoModelForCausalLM.from_pretrained(repo_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(repo_id)

# 2) Проверяем наличие спецтокенов
assert "<User>:" in tokenizer.additional_special_tokens
assert "<Bot>:"  in tokenizer.additional_special_tokens

# 3) Формируем prompt без дублирования вопроса
question = "Как скачать электронную книгу?"
prompt = f"<User>: {question} {tokenizer.eos_token} <Bot>:"

# 4) Токенизация
inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

# 5) Генерация (пример)
greedy_ids = model.generate(
    **inputs,
    max_new_tokens=30,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=False
)
print("Greedy:", tokenizer.decode(greedy_ids[0], skip_special_tokens=True))

sample_ids = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_p=0.85,
    top_k=40,
    temperature=0.9,
    repetition_penalty=1.5,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)
print("Sampled:", tokenizer.decode(sample_ids[0], skip_special_tokens=True))


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Greedy:  Как скачать электронную книгу?   Здравствуйте!  Здравствуйте.  Благодарю за обращение. Уточняю: "Как скачать электронную книгу?" иденциальность  Владимировна, уточняю. "
Sampled:  Как скачать электронную книгу?   Если вы получили электронные книги, то с помощью электронного ключа можно добавить в библиотеку. Для этого нажмите кнопку "Пожаловаться на электронное письмо" и перейдите к следующему окну:   В поле для сообщений выберите категорию ""Электронные издания""
